In [2]:
import pandas as pd
import yfinance as yf
import numpy as np 
# from scipy.stats import gmean

# Start dates - edit as required
start_date = "2023-11-11" # yyyy-mm-dd
end_date = "2024-11-11"

# Load original portfolio
portf = pd.read_csv("portfolios/fin456_portfolio_holdings_t0.csv")

# Load additional portfolio // optional, can be used to compare proposed changes.
# additional_portf = pd.read_csv("fin456_portfolio_holdings_t1.csv")

def calculate_portfolio_metrics(portf):
    # Separate cash and stock rows
    cash_row = portf[portf['TICKER'] == 'USD']
    stock_rows = portf[portf['TICKER'] != 'USD'].copy()  # Ensure this is a copy

    # Fetch data for the tickers in the portfolio
    tickers = stock_rows['TICKER'].tolist()
    data = yf.download(tickers, start=start_date, end=end_date, interval="1mo")['Adj Close']

    # Calculate monthly returns
    returns = data.pct_change().dropna()

    # Calculate total investment in each stock
    current_prices = data.iloc[-1]  # Last row gives the latest prices
    stock_rows['Investment'] = stock_rows['QUANTITY'].values * current_prices.values

    # Check if cash row is empty and handle accordingly
    if not cash_row.empty:
        cash_quantity = cash_row['QUANTITY'].iloc[0]
    else:
        cash_quantity = 0

    # Add the cash row back for total investment
    total_investment = stock_rows['Investment'].sum() + cash_quantity

    # Calculate weights
    stock_rows['Weight'] = stock_rows['Investment'] / total_investment

    # Handle cash weight separately
    cash_weight = cash_quantity / total_investment if cash_quantity > 0 else 0

    # Calculate weighted monthly returns for the portfolio
    weighted_returns = (returns * stock_rows.set_index('TICKER')['Weight']).sum(axis=1)

    # Adjust weighted returns to include cash
    weighted_returns = weighted_returns * (1 - cash_weight)

    # Expected annual return
    # expected_return = weighted_returns.mean() * 12 # using arithmetric mean
    expected_return = (1+weighted_returns).prod()**(12/weighted_returns.size) - 1 # using geometric mean

    # Portfolio variance and standard deviation (risk)
    portfolio_variance = np.dot(stock_rows.set_index('TICKER')['Weight'].T, 
                                np.dot(returns.cov() * 12, 
                                       stock_rows.set_index('TICKER')['Weight']))
    portfolio_variance *= (1 - cash_weight)**2  # Adjust for cash weight
    portfolio_std_dev = np.sqrt(portfolio_variance)

    return expected_return, portfolio_std_dev, weighted_returns, cash_weight

# Function to calculate Beta and Sharpe Ratio
def calculate_beta_sharpe(portfolio_returns, market_returns, risk_free_rate, portfolio_std_dev, expected_return):
    aligned_portfolio_returns, aligned_market_returns = portfolio_returns.align(market_returns, join='inner')

    # Ensure aligned_market_returns is a Series
    if isinstance(aligned_market_returns, pd.DataFrame):
        aligned_market_returns = aligned_market_returns.squeeze()  # Convert single-column DataFrame to Series

    cov_matrix = np.cov(aligned_portfolio_returns, aligned_market_returns)

    # Calculate beta
    beta = cov_matrix[0, 1] / cov_matrix[1, 1]

    # Sharpe Ratio
    sharpe_ratio = (expected_return - risk_free_rate) / portfolio_std_dev
    return beta, sharpe_ratio

# Function to calculate the Alpha
def calculate_alpha(portf_ret, risk_free_rate, beta, market_ret):
    # Alpha = R - Rf - beta (Rm - Rf) ... R is the portf_ret, Rf is the risk_free_rate, beta is the systematic risk of the portfolio, Rm is the market return.
    alpha = portf_ret - risk_free_rate - beta*(market_ret-risk_free_rate)
    return alpha

# Calculate metrics for original portfolio
original_metrics = calculate_portfolio_metrics(portf)

# # Download S&P 500 (or another market index) data
market_data = yf.download('^GSPC', start=start_date, end=end_date, interval="1mo")['Adj Close']

# # Calculate market monthly returns
market_returns = market_data.pct_change().dropna()

# Fetch risk-free rate (10-Year Treasury Yield)
risk_free_data = yf.download('^TNX', start=start_date, end=end_date, interval="1mo")
risk_free_rate = risk_free_data['Adj Close'].dropna().iloc[-1].item() / 100

# Calculate Beta and Sharpe Ratio for both portfolios
original_beta, original_sharpe = calculate_beta_sharpe(
    original_metrics[2], market_returns, risk_free_rate, original_metrics[1], original_metrics[0]
)

# original_alpha = calculate_alpha()

# Calculate metrics for additional portfolio # uncomment for additional portfolio
# additional_metrics = calculate_portfolio_metrics(additional_portf)

# additional_beta, additional_sharpe = calculate_beta_sharpe(  # uncomment for additional portfolio
#     additional_metrics[2], market_returns, risk_free_rate, additional_metrics[1], additional_metrics[0]
# )






[*********************100%***********************]  37 of 37 completed

37 Failed downloads:
['VHT', 'VCR', 'CVI', 'CVX', 'FDX', 'YUM', 'ADBE', 'SCHW', 'CPAY', 'LRCX', 'VGT', 'VFH', 'BRC', 'VDE', 'ADM', 'OVV', 'LMT', 'VAW', 'BBY', 'BLK', 'MPC', 'DIS', 'TEL', 'VOX', 'MOH', 'VIS', 'ATKR', 'JPM', 'GOOGL', 'MCK', 'VDC', 'MYE', 'MET', 'VPU', 'LDOS', 'JBL', 'VICI']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')


IndexError: single positional indexer is out-of-bounds

In [4]:
portf.head()

,TICKER,QUANTITY
0,ADBE,5
1,ADM,22
2,ATKR,11
3,BBY,32
4,BLK,3


In [3]:
original_metrics

NameError: name 'original_metrics' is not defined

In [10]:
aligned_portfolio_returns, aligned_market_returns = original_metrics[2].align(market_returns, join='inner')
aligned_portfolio_returns

Date
2024-01-01    0.006758
2024-02-01    0.037804
2024-03-01    0.036962
2024-04-01   -0.036994
2024-05-01    0.032165
2024-06-01    0.018229
2024-07-01    0.030247
2024-08-01    0.017712
2024-09-01    0.007633
2024-10-01   -0.009142
2024-11-01    0.037091
dtype: float64

In [11]:
aligned_market_returns

Date
2024-01-01    0.015896
2024-02-01    0.051721
2024-03-01    0.031019
2024-04-01   -0.041615
2024-05-01    0.048021
2024-06-01    0.034670
2024-07-01    0.011321
2024-08-01    0.022835
2024-09-01    0.020197
2024-10-01   -0.009897
2024-11-01    0.037075
Name: Adj Close, dtype: float64

In [17]:
# Function to calculate Beta and Sharpe Ratio
def calculate_beta_sharpe(portfolio_returns, market_returns, risk_free_rate, portfolio_std_dev, expected_return):
    aligned_portfolio_returns, aligned_market_returns = portfolio_returns.align(market_returns, join='inner')

    # Ensure aligned_market_returns is a Series
    if isinstance(aligned_market_returns, pd.DataFrame):
        aligned_market_returns = aligned_market_returns.squeeze()  # Convert single-column DataFrame to Series

    cov_matrix = np.cov(aligned_portfolio_returns, aligned_market_returns) # returns cov matrix of [[var(apr), cov(apr,amr)],[cov(amr,apr),var(amr)]]
    print(cov_matrix)
    # Calculate beta
    beta = cov_matrix[0, 1] / cov_matrix[1, 1]

    # Sharpe Ratio
    sharpe_ratio = (expected_return - risk_free_rate) / portfolio_std_dev
    return beta, sharpe_ratio

In [18]:
# Calculate Beta and Sharpe Ratio for both portfolios
original_beta, original_sharpe = calculate_beta_sharpe(
    original_metrics[2], market_returns, risk_free_rate, original_metrics[1], original_metrics[0]
)

# original_alpha = calculate_alpha()

[[0.00054272 0.00057025]
 [0.00057025 0.00072099]]


In [20]:
market_returns

Date
2024-01-01    0.015896
2024-02-01    0.051721
2024-03-01    0.031019
2024-04-01   -0.041615
2024-05-01    0.048021
2024-06-01    0.034670
2024-07-01    0.011321
2024-08-01    0.022835
2024-09-01    0.020197
2024-10-01   -0.009897
2024-11-01    0.037075
Name: Adj Close, dtype: float64

In [19]:
original_beta

0.7909302802270889

In [31]:
original_metrics[2]

Date
2024-01-01    0.006758
2024-02-01    0.037804
2024-03-01    0.036962
2024-04-01   -0.036994
2024-05-01    0.032165
2024-06-01    0.018229
2024-07-01    0.030247
2024-08-01    0.017712
2024-09-01    0.007633
2024-10-01   -0.009142
2024-11-01    0.037091
dtype: float64

In [26]:
def get_exp_ret(ticker, start_date, end_date):

    # Download S&P 500 (or another market index) data
    market_data = yf.download(ticker, start=start_date, end=end_date, interval="1mo")['Adj Close']

    # Calculate market monthly returns
    market_returns = market_data.pct_change().dropna()

    expected_return = (1+market_returns).prod()**(12/market_returns.size) - 1 # using geometric mean

    return expected_return


In [33]:
market_ret = get_exp_ret('^GSPC',start_date,end_date)

[*********************100%%**********************]  1 of 1 completed


In [35]:
original_alpha = calculate_alpha(original_metrics[0],risk_free_rate,original_beta,market_ret) 
original_alpha

-0.015539131756187458

In [1]:
import time
time.time()

1732414386.2541435

In [2]:
time.gmtime(time.time())

time.struct_time(tm_year=2024, tm_mon=11, tm_mday=24, tm_hour=2, tm_min=13, tm_sec=23, tm_wday=6, tm_yday=329, tm_isdst=0)

In [ ]:
a = time.strftime("%Y-%m-%d",time.gmtime(time.time()))

In [10]:
a = time.strptime(a,"%Y-%m-%d")

In [11]:
time.mktime(a)

1732428000.0

In [13]:
a

'2024-11-24'

In [25]:
from datetime import datetime
import pytz

type(datetime)

type

In [20]:
pytz.timezone('America/New_York')

<DstTzInfo 'America/New_York' LMT-1 day, 19:04:00 STD>

In [26]:
datetime.now(pytz.timezone('America/New_York'))

datetime.datetime(2024, 11, 23, 21, 26, 8, 342276, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>)

In [28]:
datetime.now(pytz.timezone('America/New_York')).strftime("%Y-%m-%d")

'2024-11-23'

In [30]:
end_date = datetime.now(pytz.timezone('America/New_York')).strftime("%Y-%m-%d") # yyyy-mm-dd
start_date = datetime.now(pytz.timezone('America/New_York')).replace(year=datetime.now(pytz.timezone('America/New_York')).year-1).strftime("%Y-%m-%d")
 

In [31]:
end_date

'2024-11-23'

In [33]:
type(start_date)

str

In [34]:
type("2024")

str

In [1]:
import yfinance as yf

ticker = yf.Ticker("AAPL")
data = ticker.history(period='1d')

In [2]:
nTicker = yf.Ticker("SPY")

In [14]:
nTicker

yfinance.Ticker object <SPY>

In [3]:
data = nTicker.history(period = "1y")

In [6]:
data

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,
2024-02-27 00:00:00-05:00,500.282698,500.736864,498.357382,500.509766,48854500,0.0,0.0,0.0
2024-02-28 00:00:00-05:00,498.930053,500.440674,498.564743,499.848297,56506600,0.0,0.0,0.0
2024-02-29 00:00:00-05:00,501.635411,503.284244,498.949858,501.645264,83924800,0.0,0.0,0.0
2024-03-01 00:00:00-05:00,502.533845,506.789227,502.119151,506.354797,76805900,0.0,0.0,0.0
2024-03-04 00:00:00-05:00,505.545259,507.687759,505.515610,505.811798,49799300,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2025-02-20 00:00:00-05:00,611.539978,611.679993,607.020020,610.380005,36554000,0.0,0.0,0.0
2025-02-21 00:00:00-05:00,610.159973,610.299988,599.469971,599.940002,76519800,0.0,0.0,0.0
2025-02-24 00:00:00-05:00,602.020020,603.030029,596.489990,597.210022,50737200,0.0,0.0,0.0


In [10]:
import pandas as pd 
a = pd.read_csv("portfolios/simple_portf.csv")

In [11]:
a

,TICKER,QUANTITY
0,JPM,10
1,GOOG,10
2,AMZN,10
3,STLA,10


In [21]:
tickers = a["TICKER"].to_list()

In [22]:
tickers

['JPM', 'GOOG', 'AMZN', 'STLA']

In [30]:
dat = yf.download(tickers, start="2023-11-11", interval="1mo")["Close"]

[*********************100%***********************]  4 of 4 completed


In [29]:
dat

Price            Close                                           High  \
Ticker            AMZN        GOOG         JPM       STLA        AMZN   
Date                                                                    
2023-12-01  151.940002  140.425430  165.279251  21.825058  155.630005   
2024-01-01  155.199997  141.292328  169.418518  20.552240  161.729996   
2024-02-01  176.759995  139.279556  181.901718  24.576588  177.220001   
2024-03-01  180.380005  151.714874  195.823486  26.485811  181.699997   
2024-04-01  175.000000  164.050552  187.454788  20.870445  189.770004   
2024-05-01  176.440002  173.337189  199.256943  22.270000  191.699997   
2024-06-01  193.250000  182.763306  198.893097  19.850000  199.839996   
2024-07-01  186.979996  172.726410  209.257660  16.690001  201.199997   
2024-08-01  178.500000  164.706085  222.282806  16.780001  190.600006   
2024-09-01  186.330002  166.781006  208.498901  14.050000  195.369995   
2024-10-01  186.399994  172.494308  219.435059  13.620000  195.610001   
2024-11-01  207.889999  170.296799  248.436920  13.200000  215.899994   
2024-12-01  219.389999  190.224197  238.478348  13.050000  233.000000   
2025-01-01  237.679993  205.600006  265.926575  13.130000  241.770004   
2025-02-01  214.350006  174.699997  258.790009  13.290000  242.520004   

Price                                                 Low              \
Ticker            GOOG         JPM       STLA        AMZN        GOOG   
Date                                                                    
2023-12-01  143.429650  165.852527  22.143262  142.809998  128.936712   
2024-01-01  154.644347  173.246858  21.750186  144.050003  136.360053   
2024-02-01  150.155487  182.263444  25.016457  155.619995  136.150799   
2024-03-01  152.651511  196.234098  27.618244  171.470001  131.079029   
2024-04-01  175.788376  196.449173  26.663632  166.320007  150.540091   
2024-05-01  179.305733  202.452842  23.370001  173.869995  164.309613   
2024-06-01  186.828700  199.227449  22.219999  175.919998  171.832580   
2024-07-01  192.837095  213.938418  21.049999  176.800003  165.459243   
2024-08-01  175.250219  222.955185  16.920000  151.610001  156.216909   
2024-09-01  166.950588  221.590652  16.379999  171.160004  147.837456   
2024-10-01  183.581721  224.210977  13.920000  180.250000  160.937412   
2024-11-01  183.816449  253.003333  14.230000  194.309998  165.122661   
2024-12-01  202.650103  248.964185  14.080000  209.509995  170.077056   
2025-01-01  207.080002  269.428508  13.710000  216.199997  188.660004   
2025-02-01  208.699997  280.250000  14.280000  204.160004  173.589996   

Price                                    Open                          \
Ticker             JPM       STLA        AMZN        GOOG         JPM   
Date                                                                    
2023-12-01  151.403958  20.299551  146.000000  132.842690  151.403958   
2024-01-01  159.643628  19.541474  151.539993  139.100207  164.297869   
2024-02-01  167.598681  20.917241  155.869995  143.175561  169.759296   
2024-03-01  180.151743  24.604665  176.750000  139.110170  181.549777   
2024-04-01  175.195038  20.645831  180.789993  151.286417  195.520408   
2024-05-01  185.322824  21.129999  181.639999  165.585029  189.069399   
2024-06-01  187.702543  19.590000  177.699997  173.257468  198.942268   
2024-07-01  198.735779  16.570000  193.490005  184.028695  199.463451   
2024-08-01  188.762392  15.120000  189.289993  171.559274  210.812691   
2024-09-01  198.363675  13.760000  177.550003  162.915485  219.810805   
2024-10-01  202.051908  12.740000  184.899994  168.668647  206.560845   
2024-11-01  218.043885  12.580000  199.000000  171.345597  222.411329   
2024-12-01  228.350646  12.120000  209.960007  170.127002  248.267785   
2025-01-01  237.513336  12.220000  222.029999  191.485001  239.682128   
2025-02-01  253.350006  12.390000  234.059998  202.214996  261.829987   

Price                      Volume                      

In [31]:
dat

Ticker,AMZN,GOOG,JPM,STLA
Date,,,,
2023-12-01,151.940002,140.425430,165.279251,21.825058
2024-01-01,155.199997,141.292328,169.418503,20.552240
2024-02-01,176.759995,139.279556,181.901718,24.576588
2024-03-01,180.380005,151.714874,195.823471,26.485811
2024-04-01,175.000000,164.050552,187.454788,20.870445
2024-05-01,176.440002,173.337189,199.256958,22.270000
2024-06-01,193.250000,182.763306,198.893112,19.850000
2024-07-01,186.979996,172.726410,209.257660,16.690001
2024-08-01,178.500000,164.706085,222.282806,16.780001
